# Ideal plastic Von Mises material

Author(s): Olli Väinölä <olli.vainola@student.oulu.fi>

In this notebook is an small tutorial, how to create a Von Mises material without any hardening. Equations are formulated into rate depended form.

In [1]:
# imports
using ForwardDiff
using NLsolve
# using PyPlot

Let's create a isotropic Hooke material.

In [2]:


"""
Create a isotropic Hooke material matrix C 

More information: # http://www.efunda.com/formulae/solid_mechanics/mat_mechanics/hooke_isotropic.cfm

Parameters
----------
    E: Float
        Elastic modulus
    ν: Float
        Poisson constant

Returns
-------
    Array{Float64, (6,6)}
"""
function hookeStiffnessTensor(E, ν)
    a = 1 - ν
    b = 1 - 2*ν
    c = 1 + ν
    multiplier = E / (b * c)
    return Float64[a ν ν 0 0 0;
                   ν a ν 0 0 0;
                   ν ν a 0 0 0;
                   0 0 0 b 0 0;
                   0 0 0 0 b 0;
                   0 0 0 0 0 b].*multiplier
end

# Pick material values
E = 200.0e3
ν =  0.3
C = hookeStiffnessTensor(E, ν)

6x6 Array{Float64,2}:
 2.69231e5  1.15385e5  1.15385e5  0.0        0.0        0.0      
 1.15385e5  2.69231e5  1.15385e5  0.0        0.0        0.0      
 1.15385e5  1.15385e5  2.69231e5  0.0        0.0        0.0      
 0.0        0.0        0.0        1.53846e5  0.0        0.0      
 0.0        0.0        0.0        0.0        1.53846e5  0.0      
 0.0        0.0        0.0        0.0        0.0        1.53846e5

# Defining equations for the calculation

Functions are defined for strain controller simulation

In [3]:

M = [1 0 0 0 0 0;
     0 1 0 0 0 0;
     0 0 1 0 0 0;
     0 0 0 2 0 0;
     0 0 0 0 2 0;
     0 0 0 0 0 2;]

"""
Equivalent tensile stress. 

More info can be found from: https://en.wikipedia.org/wiki/Von_Mises_yield_criterion
    Section: Reduced von Mises equation for different stress conditions

Parameters
----------
    σ: Array{Float64, 6}
        Stress in Voigt notation

Returns
-------
    Float
"""
function σₑ(σ)
    s = σ[1:6] - 1/3 * sum([σ[1], σ[2], σ[3]]) * [1 1 1 0 0 0]'
    return sqrt(3/2 * s' * M * s)[1]
end


"""
Von Mises Yield criterion

More info can be found from: http://csm.mech.utah.edu/content/wp-content/uploads/2011/10/9tutorialOnJ2Plasticity.pdf

Parameters
----------
    σ: Array{Float64, 6}
        Stress in Voigt notation
    k: Float64
        Material constant, Yield limit

Returns
-------
    Float
"""
function vonMisesYield(σ, k)
    σₑ(σ) - k
end

"""
Function for NLsolve. Inside this function are the equations which we want to find root.
Ψ is the yield function below. Functions defined here:

    dσ - C (dϵ - dλ*dΨ/dσ) = 0
                 σₑ(σ) - k = 0

Parameters
----------
    params: Array{Float64, 7}
        Array containing values from solver
    dϵ: Array{Float64, 6}
        Strain rate vector in Voigt notation
    C: Array{Float64, (6, 6)}
        Material tensor
    k: Float
        Material constant, yield limit
    Δt: Float
        time increment
    σ_begin:Array{Float64, 6}
        Stress vector in Voigt notation

Returns
-------
    Array{Float64, 7}, return values for solver
"""
function G(params, dϵ, C, k, Δt, σ_begin)

    # Creating wrapper for gradient
    yield(pars) = vonMisesYield(pars, k)
    dfdσ = ForwardDiff.gradient(yield)
    
    # Stress rate
    dσ = params[1:6]
    
    σ_tot = [vec(σ_begin); 0.0] + params
    
    # Calculating plastic strain rate
    dϵp = params[end] * dfdσ(σ_tot)
    
    # Calculating equations
    function_1 = dσ - C * (dϵ - dϵp[1:6])
    function_2 = yield(σ_tot)
    [vec(function_1); function_2]
end

"""
Function which calculates the stress. Also handles if any yielding happens

Parameters
----------
    dϵ: Array{Float64, 6}
        Strain rate vector in Voigt notation
    Δt: Float
        time increment
    σ: Array{Float64, 6}
        Last stress vector in Voigt notation
    C: Array{Float64, (6, 6)}
        Material tensor
    k: Float
        Material constant, yield limit

Returns
-------
    Tuple
    Plastic strain rate dϵᵖ and new stress vector σ
"""
function calculate_stress(dϵ, Δt, σ, C, k)
    # Test stress
    σ_tria = σ + Δt * C * dϵ
    
    # Calculating yield
    yield = vonMisesYield(σ_tria, k)

    if yield > 1
        # Yielding happened
        # Creating functions for newton: xₙ₊₁ = xₙ - df⁻¹ * f and initial values
        initial_guess = [vec(σ_tria - σ); 0.1]
        f(σ_)  = G(σ_, dϵ, C, k, Δt, σ)
        df     = ForwardDiff.jacobian(f)
        
        # Calculating root 
        result = nlsolve(not_in_place(f, df), initial_guess).zero

        σ[:]  += result[1:6]
    else
        σ = σ_tria
    end
    return σ
end

calculate_stress (generic function with 1 method)

# Defining strain history

In [8]:
steps = 10

ϵ_tot = zeros(Float64, (steps, 6))

# Adding only strain in x-axis and counting for the poisson effect
ϵ_tot[:, 1] = linspace(0, 0.002, steps)
ϵ_tot[:, 2] = linspace(0, 0.002, steps).*-ν
ϵ_tot[:, 3] = linspace(0, 0.002, steps).*-ν
#ϵ_tot[:, 2] = 1/3 * linspace(0, 0.002, steps)

linspace(-0.0,-0.0006,10)

# Simulation

Ok, we're good to go! Now we just need to define yield limit and the main loop.

This simulation is not time dependent, but since it's already defined in the equations we'll give it value 1

In [9]:
ϵ_last = zeros(Float64, (6))
ϵᵖ = zeros(Float64, (6))
σ = zeros(Float64, (6, 1))
Δt = 1.0
σy =  200.0
ss = zeros(Float64, steps)
ee = zeros(Float64, steps)
for i=2:steps
    dϵ = (reshape(ϵ_tot[i, :, :], (6, 1)) - ϵ_last) / Δt 
    σ = calculate_stress(dϵ, Δt, σ, C, σy)
    ϵ_last += dϵ * Δt
    ss[i] = σ[1]
    ee[i] = ϵ_last[1]
    sdt = [σ[1]    0   0;
           0    σ[2]   0;
           0       0 σ[3]]
    princip = eig(sdt)
    println(σ')
    println(princip)
    
end
#PyPlot.plot(ee, ss)
#PyPlot.title("Stress-Strain curve")
#PyPlot.xlabel("Strain")
#PyPlot.ylabel("Stress")
#PyPlot.ylim([0, 250])

[44.44444444444445 -3.552713678800501e-15 -3.552713678800501e-15 0.0 0.0 0.0]
([-3.552713678800501e-15,-3.552713678800501e-15,44.44444444444445],
[0.0 0.0 1.0
 1.0 0.0 0.0
 0.0 1.0 0.0])
[88.8888888888889 -7.105427357601002e-15 -7.105427357601002e-15 0.0 0.0 0.0]
([-7.105427357601002e-15,-7.105427357601002e-15,88.8888888888889],
[0.0 0.0 1.0
 1.0 0.0 0.0
 0.0 1.0 0.0])
[133.33333333333334 0.0 0.0 0.0 0.0 0.0]
([0.0,0.0,133.33333333333334],
[0.0 0.0 1.0
 1.0 0.0 0.0
 0.0 1.0 0.0])
[177.7777777777778 -6.217248937900877e-15 -7.105427357601002e-15 0.0 0.0 0.0]
([-7.105427357601002e-15,-6.217248937900877e-15,177.7777777777778],
[0.0 0.0 1.0
 0.0 1.0 0.0
 1.0 0.0 0.0])
[207.40740740740782 7.407407407407826 7.407407407407835 0.0 0.0 0.0]
([7.407407407407826,7.407407407407835,207.40740740740782],
[0.0 0.0 1.0
 1.0 0.0 0.0
 0.0 1.0 0.0])
[222.22222222222263 22.22222222222264 22.222222222222648 0.0 0.0 0.0]
([22.22222222222264,22.222222222222648,222.22222222222263],
[0.0 0.0 1.0
 1.0 0.0 0.0
 0.